In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "4g").appName('chapter_2').getOrCreate()

23/12/28 15:29:42 WARN Utils: Your hostname, quangtn933.local resolves to a loopback address: 127.0.0.1; using 192.168.1.90 instead (on interface en0)
23/12/28 15:29:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 15:29:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Download donation.zip

In [3]:
prev = spark.read.csv("./linkage/block*.csv")

23/12/28 15:29:55 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
prev

In [ ]:
prev.show(2)

In [4]:
parsed = spark.read.option("header", "true").option("nullvalue", "?").option("inferSchema", "true").csv("./linkage/block*.csv")

In [ ]:
parsed.printSchema()

In [5]:
from pyspark.sql.types import *
schema = StructType([StructField("id_1", IntegerType(), False),
                    StructField("id_2", StringType(), False),
                    StructField("cmp_fname_c1", DoubleType(), False)])

In [6]:
spark.read.schema(schema).csv("./linkage/block*.csv")

DataFrame[id_1: int, id_2: string, cmp_fname_c1: double]

In [ ]:
parsed.show(5)

In [ ]:
parsed.count()

In [ ]:
parsed.cache()

In [7]:
from pyspark.sql.functions import col

In [ ]:
parsed.groupby("is_match").count().orderBy(col("count").desc()).show()

In [ ]:
from pyspark.sql.functions import avg, stddev
parsed.agg(avg("cmp_sex"), stddev("cmp_sex")).show()

In [ ]:
parsed.createOrReplaceTempView("linkage")

In [ ]:
spark.sql("""
    SELECT is_match, COUNT(*) cnt
    FROM linkage
    GROUP BY is_match
    ORDER BY cnt DESC
    """).show()

In [8]:
summary = parsed.describe()
summary.show()

23/12/28 15:30:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|       cmp_fname_c1|      cmp_fname_c2|      cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|            5748125|            103698|           5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935| 0.7129024704437266|0.9000176718903189|0.315627819308

In [ ]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

In [24]:
matches = parsed.where("is_match=true")
match_summary = matches.describe()

In [26]:
match_summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+-------------------+-------------------+
|summary|       cmp_fname_c1|       cmp_fname_c2|
+-------+-------------------+-------------------+
|  count|              20922|               1333|
|   mean| 0.9973163859635038| 0.9898900320318176|
| stddev|0.03650667584833679|0.08251973727615237|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [25]:
misses = parsed.filter(col("is_match")== False)
miss_summary = misses.describe()
miss_summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+-------------------+-------------------+
|summary|       cmp_fname_c1|       cmp_fname_c2|
+-------+-------------------+-------------------+
|  count|            5727203|             102365|
|   mean| 0.7118634802175091| 0.8988473514090158|
| stddev|0.38908060096985553|0.27272090294010215|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [9]:
summary_p = summary.toPandas()

In [10]:
summary_p.head()

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,5749132,5749132,5748125,103698,5749132,2464,5749132,5748337,5748337,5748337,5736289
1,mean,33324.48559643438,66587.43558331935,0.7129024704437266,0.9000176718903189,0.3156278193080383,0.3184128315317443,0.955001381078048,0.22446526708507172,0.48885529849763504,0.2227485966810923,0.00552866147434343
2,stddev,23659.859374488064,23620.487613269695,0.38875835961628014,0.2713176105782334,0.3342336339615828,0.36856706620066537,0.20730111116897781,0.41722972238462636,0.4998758236779031,0.4160909629831756,0.07414914925420046
3,min,1,6,0.0,0.0,0.0,0.0,0,0,0,0,0
4,max,99980,100000,1.0,1.0,1.0,1.0,1,1,1,1,1


In [11]:
summary_p.shape

(5, 12)

In [15]:
summary_p = summary_p.set_index('summary').transpose().reset_index()
summary_p = summary_p.rename(columns={'index':'field'})
summary_p = summary_p.rename_axis(None, axis=1)

KeyError: "None of ['summary'] are in the columns"

In [16]:
summary_p

,field,count,mean,stddev,min,max
0,id_1,5749132,33324.48559643438,23659.859374488064,1,99980
1,id_2,5749132,66587.43558331935,23620.487613269695,6,100000
2,cmp_fname_c1,5748125,0.7129024704437266,0.38875835961628014,0.0,1.0
3,cmp_fname_c2,103698,0.9000176718903189,0.2713176105782334,0.0,1.0
4,cmp_lname_c1,5749132,0.3156278193080383,0.3342336339615828,0.0,1.0
5,cmp_lname_c2,2464,0.3184128315317443,0.36856706620066537,0.0,1.0
6,cmp_sex,5749132,0.955001381078048,0.20730111116897781,0,1
7,cmp_bd,5748337,0.22446526708507172,0.41722972238462636,0,1
8,cmp_bm,5748337,0.48885529849763504,0.4998758236779031,0,1
9,cmp_by,5748337,0.2227485966810923,0.4160909629831756,0,1


In [20]:
summaryT = spark.createDataFrame(summary_p)
summaryT.show()

+------------+-------+-------------------+-------------------+---+------+
|       field|  count|               mean|             stddev|min|   max|
+------------+-------+-------------------+-------------------+---+------+
|        id_1|5749132|  33324.48559643438| 23659.859374488064|  1| 99980|
|        id_2|5749132|  66587.43558331935| 23620.487613269695|  6|100000|
|cmp_fname_c1|5748125| 0.7129024704437266|0.38875835961628014|0.0|   1.0|
|cmp_fname_c2| 103698| 0.9000176718903189| 0.2713176105782334|0.0|   1.0|
|cmp_lname_c1|5749132| 0.3156278193080383| 0.3342336339615828|0.0|   1.0|
|cmp_lname_c2|   2464| 0.3184128315317443|0.36856706620066537|0.0|   1.0|
|     cmp_sex|5749132|  0.955001381078048|0.20730111116897781|  0|     1|
|      cmp_bd|5748337|0.22446526708507172|0.41722972238462636|  0|     1|
|      cmp_bm|5748337|0.48885529849763504| 0.4998758236779031|  0|     1|
|      cmp_by|5748337| 0.2227485966810923| 0.4160909629831756|  0|     1|
|     cmp_plz|5736289|0.00552866147434

In [21]:
summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: string (nullable = true)
 |-- mean: string (nullable = true)
 |-- stddev: string (nullable = true)
 |-- min: string (nullable = true)
 |-- max: string (nullable = true)



In [22]:
from pyspark.sql.types import DoubleType
for c in summaryT.columns:
    if c=='field':
        continue
    summaryT = summaryT.withColumn(c, summaryT[c].cast(DoubleType()))

In [23]:
summaryT.printSchema()

root
 |-- field: string (nullable = true)
 |-- count: double (nullable = true)
 |-- mean: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- min: double (nullable = true)
 |-- max: double (nullable = true)



In [31]:
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType

def pivot_summary(desc):
    # convert to pandas dataframe
    desc_p = desc.toPandas()
    # transpose
    desc_p = desc_p.set_index('summary').transpose().reset_index()
    desc_p = desc_p.rename(columns={'index':'field'})
    desc_p = desc_p.rename_axis(None, axis=1)
    # convert to Spark dataframe
    descT = spark.createDataFrame(desc_p)
    # convert metric column to double from string.
    for c in descT.columns:
        if c == 'field':
            continue
        else:
            descT = descT.withColumn(c, descT[c].cast(DoubleType()))
    return descT

In [32]:
match_summaryT = pivot_summary(match_summary)
match_summaryT.show()

+------------+-------+------------------+--------------------+---+-------+
|       field|  count|              mean|              stddev|min|    max|
+------------+-------+------------------+--------------------+---+-------+
|        id_1|20931.0| 34575.72117911232|   21950.31285196913|5.0|99946.0|
|        id_2|20931.0| 51259.95939037791|   24345.73345377519|6.0|99996.0|
|cmp_fname_c1|20922.0|0.9973163859635038| 0.03650667584833679|0.0|    1.0|
|cmp_fname_c2| 1333.0|0.9898900320318176| 0.08251973727615237|0.0|    1.0|
|cmp_lname_c1|20931.0|0.9970152595958817|0.043118807533945126|0.0|    1.0|
|cmp_lname_c2|  475.0| 0.969370167843852| 0.15345280740388917|0.0|    1.0|
|     cmp_sex|20931.0| 0.987291577086618| 0.11201570591216435|0.0|    1.0|
|      cmp_bd|20925.0|0.9970848267622461| 0.05391487659807981|0.0|    1.0|
|      cmp_bm|20925.0|0.9979450418160095|0.045286127452170664|0.0|    1.0|
|      cmp_by|20925.0|0.9961290322580645| 0.06209804856731055|0.0|    1.0|
|     cmp_plz|20902.0|0.9

In [34]:
miss_summaryT = pivot_summary(miss_summary)
miss_summaryT.show()

+------------+---------+--------------------+--------------------+----+--------+
|       field|    count|                mean|              stddev| min|     max|
+------------+---------+--------------------+--------------------+----+--------+
|        id_1|5728201.0|  33319.913548075565|  23665.760130330676| 1.0| 99980.0|
|        id_2|5728201.0|   66643.44259218557|  23599.551728241313|30.0|100000.0|
|cmp_fname_c1|5727203.0|  0.7118634802175091| 0.38908060096985553| 0.0|     1.0|
|cmp_fname_c2| 102365.0|  0.8988473514090158| 0.27272090294010215| 0.0|     1.0|
|cmp_lname_c1|5728201.0|  0.3131380113364304|  0.3322812130572686| 0.0|     1.0|
|cmp_lname_c2|   1989.0| 0.16295544855122532|  0.1930236663528703| 0.0|     1.0|
|     cmp_sex|5728201.0|  0.9548833918362851| 0.20755988859217375| 0.0|     1.0|
|      cmp_bd|5727412.0|  0.2216425149788421|  0.4153518275558732| 0.0|     1.0|
|      cmp_bm|5727412.0|   0.486995347986141|  0.4998308940493865| 0.0|     1.0|
|      cmp_by|5727412.0|  0.

In [35]:
match_summaryT.createOrReplaceTempView("match_desc")
miss_summaryT.createOrReplaceTempView("miss_desc")
spark.sql("""
    SELECT a.field, a.count + b.count total, a.mean - b.mean delta
    FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
    WHERE a.field NOT IN ("id_1", "id_2")
    ORDER BY delta DESC, total DESC
    """).show()

+------------+---------+--------------------+
|       field|    total|               delta|
+------------+---------+--------------------+
|     cmp_plz|5736289.0|  0.9563812499852176|
|cmp_lname_c2|   2464.0|  0.8064147192926266|
|      cmp_by|5748337.0|  0.7762059675300512|
|      cmp_bd|5748337.0|   0.775442311783404|
|cmp_lname_c1|5749132.0|  0.6838772482594513|
|      cmp_bm|5748337.0|  0.5109496938298685|
|cmp_fname_c1|5748125.0|  0.2854529057459947|
|cmp_fname_c2| 103698.0| 0.09104268062280174|
|     cmp_sex|5749132.0|0.032408185250332844|
+------------+---------+--------------------+



In [36]:
good_features = ["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]
sum_expression = " + ".join(good_features)

In [37]:
sum_expression

'cmp_lname_c1 + cmp_plz + cmp_by + cmp_bd + cmp_bm'

In [39]:
from pyspark.sql.functions import expr
scored = parsed.fillna(0, subset=good_features).withColumn('score', expr(sum_expression)). \
        select('score', 'is_match')

In [40]:
scored.show()

+-----+--------+
|score|is_match|
+-----+--------+
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
|  4.0|    true|
|  5.0|    true|
|  5.0|    true|
|  5.0|    true|
+-----+--------+
only showing top 20 rows



In [41]:
def crossTabs(scored: DataFrame, t: DoubleType) -> DataFrame:
    return scored.selectExpr(f"score >= {t} as above", "is_match"). \
            groupby("above").pivot("is_match", ("true", "false")). \
            count()

In [42]:
crossTabs(scored, 4.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20871|    637|
|false|   60|5727564|
+-----+-----+-------+



In [43]:
crossTabs(scored, 2.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20931| 596414|
|false| NULL|5131787|
+-----+-----+-------+

